In [1]:
import cython
%load_ext cython

In [27]:
%%cython
import numpy as np
cimport numpy as np

cdef dist_between(np.ndarray p, np.ndarray q):
    cdef double dx = q[0] - p[0]
    cdef double dy = q[1] - p[1]
    return np.sqrt(dx*dx + dy*dy)

def dist_pt_seg(np.ndarray p, np.ndarray u, np.ndarray v):
    cdef double seg_len = dist_between(u, v)
    cdef double t = 0.0
    t += (p[0] - u[0]) * (v[0] - u[0])
    t += (p[1] - u[1]) * (v[1] - u[1])
    t /= (seg_len*seg_len)
    if t < 0 or t > 1.0:
        return 0.0
    pt = u + t * (v-u)
    print t, pt
    return dist_between(p, pt)

In [28]:
p = np.array([0.0, 2.0])
u = np.array([-1.0, 0.0])
v = np.array([ 1.0, 0.0])
distance_point_segment(p, u, v)

0.5 [ 0.  0.]


2.0